In [1]:
import pandas as pd
import numpy as np
import os
import glob
from itertools import combinations
from collections import defaultdict
from random import sample
import random
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from scipy.stats import spearmanr, kendalltau

sns.set_context('poster')
sns.set_theme(style="whitegrid")
#pandas max row
pd.set_option('display.max_rows', 500)

plt.rcParams['font.family'] = 'Times New Roman'

In [2]:
# calculate the spearmanr of each model
def get_spearmanr(model_name, base_model, base_results, get_kendall=False):
    # get the model results
    model_results = base_results[(base_results['model_name'] == model_name) & (base_results['base_model'] == base_model)]
    gi_results = np.array(model_results['gi_rank'].tolist())


    spearman_matrix = np.zeros((len(gi_results), len(gi_results)))
    for i in range(len(gi_results)):
        for j in range(len(gi_results)):
            if get_kendall:
                spearman_matrix[i][j] = kendalltau(gi_results[i], gi_results[j])[0]
            else:
                spearman_matrix[i][j] = spearmanr(gi_results[i], gi_results[j])[0]
    # get only the upper triangle
    spearman_matrix = np.triu(spearman_matrix, k=1)
    # get mean and std
    mean = np.mean(spearman_matrix[spearman_matrix!=0])
    std = np.std(spearman_matrix[spearman_matrix!=0])
    return [model_name, base_model, mean, std]




# GILON Activity

**This file is from the COBRA Server 'benchmark_ts/6)collect_GI_rank.ipynb'**

In [7]:
with open('dataset/spearman_base_results/gilon_baseline_spearmans.pkl', 'rb') as f:
    gilon_base = pickle.load(f)

In [8]:
full_list = [['GradientShap', 'lstm'],
             ['FeatureAblation', 'lstm'],
             ['IntegratedGradients', 'lstm'],
             ['Saliency', 'lstm'],
             ['ShapleyValueSampling', 'lstm'],
             ['GradientShap', 'tcn'],
            ['FeatureAblation', 'tcn'],
            ['IntegratedGradients', 'tcn'],
            ['Saliency', 'tcn'],
            ['ShapleyValueSampling', 'tcn'],
            ['laxcat','laxcat'],
            ['FIT', 'lstm'],
            ['DynaMask', 'lstm'],
            ['DynaMask', 'tcn']]


In [9]:
gilon_spearman_results = []
for model in full_list:
    gilon_spearman_results.append(get_spearmanr(model[0], model[1], gilon_base))


In [10]:
pd.DataFrame(gilon_spearman_results, columns=['model_name', 'base_model', 'mean', 'std'])

,model_name,base_model,mean,std
0,GradientShap,lstm,0.145495,0.386609
1,FeatureAblation,lstm,0.329670,0.159194
2,IntegratedGradients,lstm,0.200440,0.333304
3,Saliency,lstm,0.542857,0.172843
4,ShapleyValueSampling,lstm,0.427253,0.197949
5,GradientShap,tcn,0.398681,0.215248
6,FeatureAblation,tcn,0.165275,0.149921
7,IntegratedGradients,tcn,0.345934,0.183195
8,Saliency,tcn,0.617582,0.135681
9,ShapleyValueSampling,tcn,0.060220,0.243731


In [11]:
gilon_kendall_results = []
for model in full_list:
    gilon_kendall_results.append(get_spearmanr(model[0], model[1], gilon_base, get_kendall=True))

pd.DataFrame(gilon_kendall_results, columns=['model_name', 'base_model', 'mean', 'std'])

,model_name,base_model,mean,std
0,GradientShap,lstm,0.129670,0.286423
1,FeatureAblation,lstm,0.235165,0.116214
2,IntegratedGradients,lstm,0.173626,0.252547
3,Saliency,lstm,0.410989,0.151569
4,ShapleyValueSampling,lstm,0.331868,0.153909
5,GradientShap,tcn,0.274725,0.168243
6,FeatureAblation,tcn,0.134066,0.105951
7,IntegratedGradients,tcn,0.235165,0.135767
8,Saliency,tcn,0.468132,0.112411
9,ShapleyValueSampling,tcn,0.046154,0.192154


# MS

In [12]:
with open('dataset/spearman_base_results/ms_baseline_spearmans.pkl', 'rb') as f:
    ms_base = pickle.load(f)

In [13]:
ms_spearman_results = []
for model in full_list:
    ms_spearman_results.append(get_spearmanr(model[0], model[1], ms_base))

In [14]:
ms_kendall_results = []
for model in full_list:
    ms_kendall_results.append(get_spearmanr(model[0], model[1], ms_base, get_kendall=True))

In [15]:
pd.DataFrame(ms_spearman_results, columns=['model_name', 'base_model', 'mean', 'std'])

,model_name,base_model,mean,std
0,GradientShap,lstm,-2.857143e-02,0.633229
1,FeatureAblation,lstm,7.028571e-01,0.141825
2,IntegratedGradients,lstm,2.857143e-02,0.481494
3,Saliency,lstm,2.342857e-01,0.325464
4,ShapleyValueSampling,lstm,7.028571e-01,0.204121
5,GradientShap,tcn,1.885714e-01,0.367496
6,FeatureAblation,tcn,4.742857e-01,0.273809
7,IntegratedGradients,tcn,1.885714e-01,0.367496
8,Saliency,tcn,2.342857e-01,0.290475
9,ShapleyValueSampling,tcn,2.571429e-01,0.374711


In [16]:
pd.DataFrame(ms_kendall_results, columns=['model_name', 'base_model', 'mean', 'std'])

,model_name,base_model,mean,std
0,GradientShap,lstm,-0.040000,0.542586
1,FeatureAblation,lstm,0.546667,0.170750
2,IntegratedGradients,lstm,0.013333,0.430813
3,Saliency,lstm,0.200000,0.230940
4,ShapleyValueSampling,lstm,0.546667,0.267997
5,GradientShap,tcn,0.173333,0.314395
6,FeatureAblation,tcn,0.360000,0.258543
7,IntegratedGradients,tcn,0.173333,0.314395
8,Saliency,tcn,0.200000,0.304047
9,ShapleyValueSampling,tcn,0.253333,0.338362
